In [4]:
import pandas as pd
df= pd.read_csv('cleaned_holiday_package_data.csv')
df.head()

,ProdTaken,Age,TypeofContact,CityTier,DurationOfPitch,Occupation,Gender,NumberOfFollowups,ProductPitched,PreferredPropertyStar,MaritalStatus,NumberOfTrips,Passport,PitchSatisfactionScore,OwnCar,Designation,MonthlyIncome,TotalNumberOfVisitors
0,1,41.0,Self Enquiry,3,6.0,Salaried,Female,3.0,Deluxe,3.0,Unmarried,1.0,1,2,1,Manager,20993.0,3.0
1,0,49.0,Company Invited,1,14.0,Salaried,Male,4.0,Deluxe,4.0,Divorced,2.0,0,3,1,Manager,20130.0,5.0
2,1,37.0,Self Enquiry,1,8.0,Free Lancer,Male,4.0,Basic,3.0,Unmarried,7.0,1,3,0,Executive,17090.0,3.0
3,0,33.0,Company Invited,1,9.0,Salaried,Female,3.0,Basic,3.0,Divorced,2.0,1,5,1,Executive,17909.0,3.0
4,0,36.0,Self Enquiry,1,8.0,Small Business,Male,3.0,Basic,4.0,Divorced,1.0,0,5,1,Executive,18468.0,2.0


### Train test split and model training

In [5]:
from sklearn.model_selection import train_test_split
X = df.drop('ProdTaken', axis=1)
y = df['ProdTaken']

In [6]:
y.value_counts()

ProdTaken
0    3968
1     920
Name: count, dtype: int64

In [7]:
# Split the dataset into training and testing sets
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

In [8]:
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((3910, 17), (978, 17), (3910,), (978,))

In [9]:
# create column transformer with one-hot encoding for categorical features and standard scaling for numerical features
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
categorical_features = X.select_dtypes(include=['object']).columns
numerical_features = X.select_dtypes(exclude=['object']).columns
numerical_features, categorical_features

(Index(['Age', 'CityTier', 'DurationOfPitch', 'NumberOfFollowups',
        'PreferredPropertyStar', 'NumberOfTrips', 'Passport',
        'PitchSatisfactionScore', 'OwnCar', 'MonthlyIncome',
        'TotalNumberOfVisitors'],
       dtype='object'),
 Index(['TypeofContact', 'Occupation', 'Gender', 'ProductPitched',
        'MaritalStatus', 'Designation'],
       dtype='object'))

In [10]:
numeric_transform=StandardScaler()
categorical_transform=OneHotEncoder(drop='first')
preprocessor=ColumnTransformer(
    transformers=[
        ("OneHotEncoder",categorical_transform,categorical_features),
        ("StandardScaler",numeric_transform,numerical_features)
    ]
)

preprocessor

,transformers,"[('OneHotEncoder', ...), ('StandardScaler', ...)]"
,remainder,'drop'
,sparse_threshold,0.3
,n_jobs,None
,transformer_weights,None
,verbose,False
,verbose_feature_names_out,True
,force_int_remainder_cols,'deprecated'
,categories,'auto'
,drop,'first'
,sparse_output,True


In [11]:
X_train =preprocessor.fit_transform(X_train)
X_test =preprocessor.transform(X_test)

In [12]:
pd.DataFrame(X_train)

,0,1,2,3,4,5,6,7,8,9,...,16,17,18,19,20,21,22,23,24,25
0,1.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,1.0,...,1.477328,-1.116889,-2.708464,1.775141,-0.654754,-0.643510,0.670552,-1.279276,-0.229477,-1.477093
1,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,...,-0.711099,-0.879814,-0.708068,-0.729659,-0.654754,-0.643510,1.401078,-1.279276,-0.959931,0.648834
2,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,...,-0.711099,-0.642739,0.292129,-0.729659,-0.654754,-0.643510,-0.790500,0.781692,0.541936,-0.059809
3,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,...,-0.711099,1.253859,0.292129,0.522741,1.524972,-0.643510,0.670552,-1.279276,-0.351596,1.357476
4,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,-0.711099,-1.116889,-0.708068,1.775141,0.435109,-0.643510,-0.790500,0.781692,-1.121876,-0.768451
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3905,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,...,1.477328,0.661172,-0.708068,1.775141,-1.199685,-0.643510,-1.521026,0.781692,0.901878,0.648834
3906,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,-0.711099,-1.116889,0.292129,1.775141,-1.199685,-0.643510,0.670552,-1.279276,1.290887,-0.768451
3907,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,1.477328,-0.642739,0.292129,-0.729659,-0.654754,-0.643510,-0.790500,0.781692,-0.058094,0.648834
3908,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,...,-0.711099,-0.642739,0.292129,1.775141,2.069903,-0.643510,1.401078,0.781692,-0.161150,1.357476


### Apply Random Forest Classifier model

In [13]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, roc_auc_score, RocCurveDisplay, precision_score, recall_score


In [14]:
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
models={
    "Random Forest Classifier":RandomForestClassifier(random_state=42),
    "Logistic Regression":LogisticRegression(random_state=42),
    "Decission Tree Classifier":DecisionTreeClassifier(random_state=42)
    }

In [15]:
for i in range (len(models)):
    model=list(models.values())[i]
    model.fit(X_train,y_train)
    y_train_pred=model.predict(X_train)
    y_test_pred=model.predict(X_test)
    
    # training set performance
    print(f"----{list(models.keys())[i]}----")
    print("Training Set Performance")
    print("Accuracy:", accuracy_score(y_train, y_train_pred))
    print("Precision:", precision_score(y_train, y_train_pred))
    print("Recall:", recall_score(y_train, y_train_pred))
    print("ROC AUC Score:", roc_auc_score(y_train, y_train_pred))
    print("\n")
    # testing set performance
    print("Testing Set Performance")
    print("Accuracy:", accuracy_score(y_test, y_test_pred))
    print("Precision:", precision_score(y_test, y_test_pred))
    print("Recall:", recall_score(y_test, y_test_pred))
    print("ROC AUC Score:", roc_auc_score(y_test, y_test_pred))
    print("\n\n")

----Random Forest Classifier----
Training Set Performance
Accuracy: 1.0
Precision: 1.0
Recall: 1.0
ROC AUC Score: 1.0


Testing Set Performance
Accuracy: 0.9355828220858896
Precision: 0.9763779527559056
Recall: 0.6739130434782609
ROC AUC Score: 0.8350673529733873



----Logistic Regression----
Training Set Performance
Accuracy: 0.8508951406649616
Precision: 0.7204610951008645
Recall: 0.33967391304347827
ROC AUC Score: 0.6545565532451165


Testing Set Performance
Accuracy: 0.8312883435582822
Precision: 0.6266666666666667
Recall: 0.2554347826086957
ROC AUC Score: 0.6100851494907459



----Decission Tree Classifier----
Training Set Performance
Accuracy: 1.0
Precision: 1.0
Recall: 1.0
ROC AUC Score: 1.0


Testing Set Performance
Accuracy: 0.908997955010225
Precision: 0.7714285714285715
Recall: 0.7336956521739131
ROC AUC Score: 0.8416589092103821





In [16]:
# hyper parameter tuning 

rf_params={
    'n_estimators':[100,200,300,400,500],
    'max_depth':[None,10,20,30,40,50],
    'min_samples_split':[2,5,10],
    'min_samples_leaf':[1,2,4]}
rf_params

{'n_estimators': [100, 200, 300, 400, 500],
 'max_depth': [None, 10, 20, 30, 40, 50],
 'min_samples_split': [2, 5, 10],
 'min_samples_leaf': [1, 2, 4]}

In [17]:
#model list for hyperparameter tuning
randomCv_models=[('RF',RandomForestClassifier(),rf_params)]

In [18]:
from sklearn.model_selection import RandomizedSearchCV
model_param={}
for name, model, params in randomCv_models:
    print(f"Tuning hyperparameters for {name}")
    random_search=RandomizedSearchCV(model, param_distributions=params, n_iter=10, cv=5, n_jobs=-1, verbose=2, random_state=42)
    random_search.fit(X_train,y_train)
    model_param[name]=random_search.best_params_

for model_name in model_param:
    print(f"---------Best params------")
    print(model_param[model_name])

Tuning hyperparameters for RF
Fitting 5 folds for each of 10 candidates, totalling 50 fits
---------Best params------
{'n_estimators': 300, 'min_samples_split': 2, 'min_samples_leaf': 1, 'max_depth': 30}


In [19]:
models={"RandomForestClassifier": RandomForestClassifier(**model_param['RF'])}
for i in range (len(models)):
    model=list(models.values())[i]
    model.fit(X_train,y_train)
    y_train_pred=model.predict(X_train)
    y_test_pred=model.predict(X_test)
    
    # training set performance
    print(f"----{list(models)[i]}----")
    print("Training Set Performance")
    print("Accuracy:", accuracy_score(y_train, y_train_pred))
    print("Precision:", precision_score(y_train, y_train_pred))
    print("Recall:", recall_score(y_train, y_train_pred))
    print("ROC AUC Score:", roc_auc_score(y_train, y_train_pred))
    print("\n")
    # testing set performance
    print("Testing Set Performance")
    print("Accuracy:", accuracy_score(y_test, y_test_pred))
    print("Precision:", precision_score(y_test, y_test_pred))
    print("Recall:", recall_score(y_test, y_test_pred))
    print("ROC AUC Score:", roc_auc_score(y_test, y_test_pred))
    print("\n\n")

----RandomForestClassifier----
Training Set Performance
Accuracy: 1.0
Precision: 1.0
Recall: 1.0
ROC AUC Score: 1.0


Testing Set Performance
Accuracy: 0.9376278118609407
Precision: 0.9555555555555556
Recall: 0.7010869565217391
ROC AUC Score: 0.8467651407293835





In [20]:
from sklearn.ensemble import RandomForestClassifier
import joblib

best_model = RandomForestClassifier(**model_param['RF'])
best_model.fit(X_train, y_train)

joblib.dump(best_model, 'best_holiday_package_model.pkl')
joblib.dump(preprocessor, 'preprocessor.pkl')


['preprocessor.pkl']